# Installing Spotipy

In [ ]:
##!conda install -c conda-forge spotipy

## Loading credentials from the config file

#### Make sure that you have stored your spotify client_id and client_secret in a separate config.py file
#### Once that's done, we import it

In [ ]:
import config

## Starting with Spotify API

In [ ]:
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))



results = sp.search(q="daddy cool",limit=5,market="GB")
results


In [ ]:
import pprint

In [ ]:
pprint.pprint(results)

In [ ]:
pprint.pprint(results["tracks"]["items"][0]["id"])

# Understanding the json

In [ ]:
print("The json file has the following keys: ",list(results.keys())) # We can see that we only have tracks
print("The 'tracks' key has the following child keys: ",list(results["tracks"].keys())) # Let's check the values
print("The query we made is: ",results["tracks"]["href"]) # Query we have searched 
print("The song's info is contained in: ",results["tracks"]["items"]) #items (actual tracks)
print("The limit of the query we've made is: ",results["tracks"]["limit"]) #Limit we have chosen
print("The next page if any: ",results["tracks"]["next"]) #link to the next page (next 50 tracks)
print("The starting webpage: ",results["tracks"]["offset"]) # Actual offset (starting point)
print("Starting webpage: ",results["tracks"]["previous"]) #Previous search
print("Total number of results: ",results["tracks"]["total"]) # Number of matches

## Other Info

In [ ]:
results["tracks"]["items"][0]["artists"] # Track artists
results["tracks"]["items"][0]["id"] # Track ID
results["tracks"]["items"][0]["name"] # Track name
results["tracks"]["items"][0]["popularity"] # Popularity index
results["tracks"]["items"][0]["uri"] # Basically ID

In [ ]:
len(results['tracks']["items"])

## Getting the track_id

In [ ]:
track_id=results["tracks"]["items"][0]["id"]
track_id

In [ ]:
for item in results['tracks']['items']:
    print("The name of song is: '{}' and the id is: {}".format(item['name'],item["id"]))

## Embedded track player

In [ ]:
from IPython.display import IFrame

track_id = '3WMbD1OyfKuwWDWMNbPQ4g'
#track_id= 'spotify:track:3hgl7EQwTutSm6PESsB7gZ'
IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [ ]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [ ]:
play_song('3WMbD1OyfKuwWDWMNbPQ4g')

# Getting the Audio features of a song

In [ ]:
id_ = results["tracks"]["items"][0]["id"]

In [ ]:
sp.audio_features(id_ )

In [ ]:
play_song('2TVxnKdb3tqe1nhQWwwZCO')

In [ ]:
sp.audio_features("3WMbD1OyfKuwWDWMNbPQ4g")

## Building a Data frame of audio features

In [ ]:
sp.audio_features(results["tracks"]["items"][0]["uri"])

In [ ]:
results

In [ ]:
list_of_songs=[]
for i in results["tracks"]["items"]:
    list_of_songs.append(i["id"])

song_features = sp.audio_features(list_of_songs)

df=pd.DataFrame(song_features)    
df=df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

df

## Searching a playlist

In [ ]:
results = sp.search(q="down jay sean")
play_song(results["tracks"]["items"][0]["id"])

In [ ]:
playlist = sp.user_playlist_tracks("spotify", "4ebKOuGNfJ5g8RdtbEBHxe",market="GB")

In [ ]:
playlist

In [ ]:
type(playlist)

In [ ]:
playlist.keys()

In [ ]:
len(playlist["items"])

## Extracting a song from playlist

In [ ]:
pprint.pprint(playlist)

In [ ]:
playlist["items"][0]["track"]["id"]

In [ ]:
play_song(playlist["items"][0]["track"]["id"])

In [ ]:
print(list(playlist.keys())) # Let's look at items and total:
print("Total number of songs in the playlist: ",playlist["total"]) #  Let's check items:
len(playlist["items"]) # It is limited to 100 tracks, we will have to fix it:

## Extracting the songs of a playlist

Pagination using "next".
When you collect songs from a playlist using sp.playlist_tracks, you're limited by the limit parameter, which has a maximum (and default) value of 100. When the playlist has more than 100 songs, you have to collect them by navigating through the "pages" of the results.

The parameter offset allows you to retrieve resuls starting at a certain position: if you start at position 101, you'd get the next "page" of results. An offset of 201 would give you the third page, and so on.

The function sp.next() does the same, but in a simpler way: it can be used on the results from any request to directly retrieve the results for the next page.

We can check whether there's a next page or not by accessing the key next on the results from any request.

In [ ]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
spanish_tracks = get_playlist_tracks("Spotify","37i9dQZF1EQpj7X7UK8OOF")

spanish_ids = []
for i in spanish_tracks:
    spanish_ids.append(i['track']['id'])
 
af = sp.audio_features(spanish_ids)

In [ ]:
len(spanish_tracks)

In [ ]:
spanish_ids = []
for i in spanish_tracks:
    spanish_ids.append(i['track']['id'])

In [ ]:
af = sp.audio_features(spanish_ids)

In [ ]:
af

In [ ]:
spanish_df = pd.DataFrame(af)
spanish_df.head()

### Chopping a big playlist into chunks

In [ ]:
big_playlist_tracks = get_playlist_tracks("Spotify","2s1KwThY09NjEkmzeTsZWe")

In [ ]:
bp_ids = []
for i in big_playlist_tracks:
    bp_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
#Here comes the code that chops the large list into smaller lists of 100 ids each
bp_ids_chopped = [bp_ids[i:i+chunk_size] for i in range(0,len(bp_ids), 100)]


In [ ]:
bp_ids_chopped

In [ ]:
af_list = []
for i in bp_ids_chopped:
    af_list.extend(sp.audio_features(i))
pd.DataFrame(af_list)

### Example with rock

In [ ]:
rock = get_playlist_tracks('Spotify','37i9dQZF1DWXRqgorJj26U')

In [ ]:
rock_ids =[]
for i in rock:
    rock_ids.append(i['track']['id'])

In [ ]:
len(rock_ids)

In [ ]:
chunk_size = 100
rock_ids_chopped = [rock_ids[i:i+chunk_size] for i in range(0,len(rock_ids), 100)]

In [ ]:
rock_af = []
for i in rock_ids_chopped:
    rock_af.extend(sp.audio_features(i))

In [ ]:
rock_df = pd.DataFrame(rock_af)
rock_df

## Optional(Extra)

## Getting the artists of the playlist 

In [ ]:
def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

In [ ]:
def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(set(artist for subset in [get_artists_from_track(track["track"]) for track in tracks_from_playlist] for artist in subset))

In [ ]:
get_artists_from_playlist("4rnleEAOdmFAbRcNCgZMpY")

# Getting albums 

In this section we will work with albums to extract information. We will start by extracting all the albums of an artist.

In [ ]:
def get_albums_from_artist(artist_id):
    results = sp.artist_albums(artist_id, limit = 50,country="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Same for albums ids
def get_album_ids_from_artist(artist_id):
    results = sp.artist_albums(artist_id, limit = 50)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return [track["id"] for track in tracks]

Example: Coldplay

In [ ]:
coldplay_id = "4gzpq5DPGxSnKTe4SA8HAU"
coldplay_albums = get_albums_from_artist(coldplay_id)
coldplay_album_ids = get_album_ids_from_artist(coldplay_id)

# Check artists that played with coldplay
set([artist["name"] for track in coldplay_albums for artist in track["artists"]])

## Getting the songs of a given album

In [61]:
def get_track_ids_from_albums(album_ids):
    return list(set([i["id"] for j in album_ids for i in sp.album(j)["tracks"]["items"]]))

In [ ]:
coldplay_songs = get_track_ids_from_albums(coldplay_album_ids)

len(coldplay_songs)